In [ ]:
!pip install pyspark
%matplotlib inline
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
import pandas as pd

     |████████████████████████████████| 204.2MB 72kB/s 
     |████████████████████████████████| 204kB 47.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=8c2f2e257c83484ba80c25b59d6d9fb66573aa0a52dc4f715e8a061fd09e65a1
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


## Before

### Hashtag, cleaning

In [ ]:
import pandas as pd
before = pd.read_csv("tweets.csv")
before

,user,id,location,date,favourites_count,text,retweet
0,PatsFan876,2.999235e+09,NaN,1/26/15,359685,Busting stereotypes here... I'm a truck drivin...,True
1,Yasu_Al_Masih,8.830000e+17,Nazareth,7/7/17,15736,I COULDN'T RESIST POSTING THIS ONE!🤣😂🤣😂🤣🤣🤣 🤦‍♀...,True
2,JBK11,1.503627e+07,Canada,6/7/08,43347,Busting stereotypes here... I'm a truck drivin...,True
3,SoyElG,3.209835e+07,NaN,4/16/09,432,The dismissive hand wave is what’s most disres...,False
4,cynthianatalie,2.110394e+07,Los Angeles,2/17/09,4952,"Inside #DonaldTrump's ""serial bad behavior"" at...",True
...,...,...,...,...,...,...,...
97195,Kimberl93935319,9.230000e+17,NaN,10/24/17,14605,@Amy_Siskind #DEMFOCUS2020 We need to focus re...,True
97196,sharknadoalert,1.280000e+18,NaN,7/16/20,6375,"""Missed opportunities add up to a losing strat...",True
97197,BereniceJB_,4.312218e+08,Orange County,12/8/11,29087,The Gun Lobby is calling on voters to vote for...,True
97198,1989Mckoy,8.440000e+17,NaN,3/22/17,135684,Please vote #registertovote if you have yet to...,False


In [ ]:
before.dtypes

user                 object
id                  float64
location             object
date                 object
favourites_count      int64
text                 object
retweet                bool
dtype: object

In [ ]:
# Pandas to Spark
from pyspark.sql.types import *
# mySchema=StructType([StructField("Country", StringType())\
#                       ,StructField("Org", StringType())])

mySchema = StructType([StructField("user", StringType())\
                      ,StructField("id", FloatType())\
                      ,StructField("location", StringType())\
                      ,StructField("date", StringType())\
                      ,StructField("favourites_count", StringType())\
                      ,StructField("text", StringType())\
                      ,StructField("retweet", StringType())])
df_before = spark.createDataFrame(before,schema=mySchema)
df_before.show()

+---------------+-------------+--------------------+--------+----------------+--------------------+-------+
|           user|           id|            location|    date|favourites_count|                text|retweet|
+---------------+-------------+--------------------+--------+----------------+--------------------+-------+
|     PatsFan876| 2.99923456E9|                 NaN| 1/26/15|          359685|Busting stereotyp...|   true|
|  Yasu_Al_Masih| 8.8299999E17|            Nazareth|  7/7/17|           15736|I COULDN'T RESIST...|   true|
|          JBK11|  1.5036271E7|              Canada|  6/7/08|           43347|Busting stereotyp...|   true|
|         SoyElG|  3.2098352E7|                 NaN| 4/16/09|             432|The dismissive ha...|  false|
| cynthianatalie|  2.1103938E7|         Los Angeles| 2/17/09|            4952|Inside #DonaldTru...|   true|
|  jaynemikulski|  7.1986381E8| Western Springs, IL|10/17/13|          103327|Is #DonaldTrump a...|   true|
|       Shrinki2|1.21999996E

In [ ]:
df_before.count()

97200

In [ ]:
# drop columns
df_before = df_before.drop('id', 'date')
# change the data type
df_before = df_before.withColumn("favourites_count", df_before['favourites_count'].cast(IntegerType()))
# # drop rows which tweet is retweet
df_before = df_before.filter(df_before['retweet'] == 'false')

In [ ]:
# check if there are duplicate rows
if df_before.distinct().count() != df_before.count():
  print('There are duplicate rows.')
else:
  print('No duplicated.')

There are duplicate rows.


In [ ]:
# remove duplicate rows
df_before = df_before.dropDuplicates()
df_before.count()

22987

In [ ]:
df_before.show(4)

+---------------+--------------------+----------------+--------------------+-------+
|           user|            location|favourites_count|                text|retweet|
+---------------+--------------------+----------------+--------------------+-------+
|    TerinaKelso|Jacksonville, Flo...|            1865|SAY IT AGAIN FOR ...|  false|
| BadTasteMedia7|      Snellville, Ga|               2|I appreciate Chri...|  false|
|Horizonshealth2|                 NaN|              28|FOODS THAT MAKE Y...|  false|
|  maskshirt2021|                 NaN|             125|Without a mask he...|  false|
+---------------+--------------------+----------------+--------------------+-------+
only showing top 4 rows



In [ ]:
# extract the hashtage
import re
#extracting hashtags as a means of possible cluster validation  
def extraxt_hashtags(text):
  hashtags = re.findall(r'\B#\w*[a-zA-Z]+\w*', text)
  return hashtags

In [ ]:
from pyspark.sql.functions import udf, expr, concat, col
from pyspark.sql import types
func_hash = udf(extraxt_hashtags, types.StringType())
df_before = df_before.withColumn("hashtage", func_hash("text"))

In [ ]:
# clean tweet
from string import punctuation
def clean_tweet(tweet):
    # Remove hyperlinks
    tweet = re.sub("https?://[A-Za-z0-9./]*","",tweet)
    # Remove hashtags
    tweet = re.sub(r'#\w*', '', tweet)
    # Remove tickers
    tweet = re.sub(r'\$\w*', '', tweet)
    #@user -> at_user
    tweet = re.sub("@[\w]*","",tweet)
    # To lowercase
    tweet = tweet.lower()
    # Remove Punctuation and split 's, 't, 've with a space for filter
    tweet = re.sub(r'[' + punctuation.replace('@', '') + ']+', ' ', tweet)
    # Remove words: I, a , am, me (2 or less letters)
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet)
    # Remove whitespace (including new line characters)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    # Remove single space remaining at the front of the tweet.
    tweet = tweet.lstrip(' ') 
    # Remove emojis or other. special characters
    tweet = ''.join(c for c in tweet if c <= '\uFFFF') 
    return tweet  

In [ ]:
func_clean = udf(clean_tweet, types.StringType())
df_before = df_before.withColumn("clean_text", func_clean("text"))

### sentiment analysis for score

In [ ]:
df_before.printSchema()

root
 |-- user: string (nullable = true)
 |-- location: string (nullable = true)
 |-- favourites_count: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- retweet: string (nullable = true)
 |-- hashtage: string (nullable = true)
 |-- clean_text: string (nullable = true)



In [ ]:
from textblob import TextBlob

def getSentiment(text):
    temp = TextBlob(text).sentiment[0]
    if temp == 0.0:
        return 0 # Neutral
    elif temp >= 0.0:
        return 1 # Positive
    else:
        return -1 # Negative

sentiment = udf(getSentiment)
df_before = df_before.withColumn("score", sentiment(df_before['clean_text']))
df_before.show(4)

+---------------+--------------------+----------------+--------------------+-------+--------------------+--------------------+-----+
|           user|            location|favourites_count|                text|retweet|            hashtage|          clean_text|score|
+---------------+--------------------+----------------+--------------------+-------+--------------------+--------------------+-----+
|    TerinaKelso|Jacksonville, Flo...|            1865|SAY IT AGAIN FOR ...|  false|[#trump, #maga, #...|say again for the...|    0|
| BadTasteMedia7|      Snellville, Ga|               2|I appreciate Chri...|  false|[#badtastemedia, ...|appreciate christ...|   -1|
|Horizonshealth2|                 NaN|              28|FOODS THAT MAKE Y...|  false|[#DonaldTrump, #T...|foods that make y...|    0|
|  maskshirt2021|                 NaN|             125|Without a mask he...|  false|[#COVIDIOTS, #Don...|without mask stan...|    1|
+---------------+--------------------+----------------+--------------

In [ ]:
df_before.toPandas().to_csv('before.csv')

### tfidf

In [ ]:
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import IDF
import requests
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer

tokenizer = Tokenizer()\
  .setInputCol("clean_text")\
  .setOutputCol("words")

# stopwords object
# we obtain the stop words from a website
stop_words = requests.get('http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words').text.split()
stop_words[0:10]
sw_filter = StopWordsRemover()\
\
  .setStopWords(stop_words)\
  .setCaseSensitive(False)\
  .setInputCol("words")\
  .setOutputCol("filtered")

# we will remove words that appear in 5 docs or less
cv = CountVectorizer()\
  .setInputCol("filtered")\
  .setOutputCol("tf")

# TF-IDF object
idf = IDF().\
    setInputCol('tf').\
    setOutputCol('tfidf')

# build the pipeline
pipe1 = Pipeline(stages=[tokenizer, sw_filter, cv]).fit(df_before)
tweets_pipe = Pipeline(stages=[pipe1, idf]).fit(df_before)

In [ ]:
tweets_before_df = tweets_pipe.transform(df_before)

In [ ]:
tweets_before_df.show(4)

+---------------+--------------------+----------------+--------------------+-------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|           user|            location|favourites_count|                text|retweet|            hashtage|          clean_text|score|               words|            filtered|                  tf|               tfidf|
+---------------+--------------------+----------------+--------------------+-------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|    TerinaKelso|Jacksonville, Flo...|            1865|SAY IT AGAIN FOR ...|  false|[#trump, #maga, #...|say again for the...|    0|[say, again, for,...|  [say, people, row]|(20493,[9,50,3185...|(20493,[9,50,3185...|
| BadTasteMedia7|      Snellville, Ga|               2|I appreciate Chri...|  false|[#badtastemedia, ...|appreciate christ...|   -1|

## After

### Hashtag, cleaning

In [ ]:
df_after = pd.read_csv("tweet_after.csv")
df_after

,user,id,location,date,favourites_count,text,retweet
0,Hskers62,9.081270e+17,United States,9/14/17,197954.0,Donald Trump is the only president in history ...,True
1,JMZJEWELRY,8.826030e+17,Here and there,7/5/17,3652.0,Joe Biden got 306 Electoral College votes by w...,True
2,lrogers66821,1.004390e+18,"Cincinnati, OH",6/6/18,369957.0,"Deutsche Bank, which funded Auschwitz, Donald ...",True
3,LizNYC66,8.927390e+17,"Manhattan, NY",8/2/17,33175.0,The Biden administration is sure to come under...,True
4,thia89837769,1.214570e+18,"Ottawa, Ontario",1/7/20,5580.0,Donald Trump is the only president in history ...,True
...,...,...,...,...,...,...,...
45389,Richardson9Mike,1.219890e+18,NaN,1/22/20,21.0,Should Nancy go to jail? #Elections2020 #Elect...,False
45390,suzyqbooks,3.720740e+07,"Salt Lake City, UT",5/2/09,8759.0,"VOTE UPDATE:\n\nGeorgia: Biden +12,337 \nArizo...",True
45391,MSZ006FD3S3121,1.041590e+18,NaN,9/17/18,47294.0,"VOTE UPDATE:\n\nGeorgia: Biden +12,337 \nArizo...",True
45392,FjellVeitje,1.297540e+18,Noreg,8/23/20,2731.0,I think this is a vote we all can agree on... ...,True


In [ ]:
df_after.dtypes

user                 object
id                  float64
location             object
date                 object
favourites_count    float64
text                 object
retweet              object
dtype: object

In [ ]:
from pyspark.sql.types import *
# mySchema=StructType([StructField("Country", StringType())\
#                       ,StructField("Org", StringType())])

mySchema = StructType([StructField("user", StringType())\
                      ,StructField("id", FloatType())\
                      ,StructField("location", StringType())\
                      ,StructField("date", StringType())\
                      ,StructField("favourites_count", StringType())\
                      ,StructField("text", StringType())\
                      ,StructField("retweet", StringType())])
tweets_after = spark.createDataFrame(df_after,schema=mySchema)
tweets_after.show(5)

+------------+-------------+---------------+-------+----------------+--------------------+-------+
|        user|           id|       location|   date|favourites_count|                text|retweet|
+------------+-------------+---------------+-------+----------------+--------------------+-------+
|    Hskers62| 9.0812699E17|  United States|9/14/17|        197954.0|Donald Trump is t...|   true|
|  JMZJEWELRY|   8.82603E17|Here and there | 7/5/17|          3652.0|Joe Biden got 306...|   true|
|lrogers66821|1.00438999E18| Cincinnati, OH| 6/6/18|        369957.0|Deutsche Bank, wh...|   true|
|    LizNYC66| 8.9273898E17|  Manhattan, NY| 8/2/17|         33175.0|The Biden adminis...|   true|
|thia89837769|1.21457002E18|Ottawa, Ontario| 1/7/20|          5580.0|Donald Trump is t...|   true|
+------------+-------------+---------------+-------+----------------+--------------------+-------+
only showing top 5 rows



In [ ]:
tweets_after.count()

45394

In [ ]:
# drop columns
tweets_after = tweets_after.drop('id', 'date')
# change the data type
tweets_after = tweets_after.withColumn("favourites_count", tweets_after['favourites_count'].cast(IntegerType()))
# # drop rows which tweet is retweet
tweets_after = tweets_after.filter(tweets_after['retweet'] == 'false')

In [ ]:
# check if there are duplicate rows
if tweets_after.distinct().count() != tweets_after.count():
  print('There are duplicate rows.')
else:
  print('No duplicated.')

There are duplicate rows.


In [ ]:
# remove duplicate rows
tweets_after = tweets_after.dropDuplicates()
tweets_after.count()

15860

In [ ]:
tweets_after = tweets_after.withColumn("hashtage", func_hash("text"))

In [ ]:
tweets_after = tweets_after.withColumn("clean_text", func_clean("text"))

### sentiment analysis for score

In [ ]:
tweets_after = tweets_after.withColumn("score", sentiment(tweets_after['clean_text']))
tweets_after.show(4)

+---------------+----------------+----------------+--------------------+-------+------------+--------------------+-----+
|           user|        location|favourites_count|                text|retweet|    hashtage|          clean_text|score|
+---------------+----------------+----------------+--------------------+-------+------------+--------------------+-----+
|nicholasmcardle|White House -TBC|             418|@realDonaldTrump ...|  false|          []|trump english can...|    1|
|     seanieinaz|             NaN|              38|@realDonaldTrump ...|  false|          []|can donald trump ...|   -1|
|    packard1963|  Northumberland|           13002|@KayBurley #mikep...|  false|[#mikepence]|saying more less ...|    1|
| mikepence_smom|   United States|             801|@MSarelson @Angel...|  false|          []|put his ass jail ...|   -1|
+---------------+----------------+----------------+--------------------+-------+------------+--------------------+-----+
only showing top 4 rows



In [ ]:
tweets_after.toPandas().to_csv('after.csv')

### idf

In [ ]:
pipe3 = Pipeline(stages=[tokenizer, sw_filter, cv]).fit(tweets_after)
tweets_pipe_3 = Pipeline(stages=[pipe2, idf]).fit(tweets_after)

In [ ]:
tweets_after_df = tweets_pipe_2.transform(tweets_after)

## Random Forest

### Before

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.feature import VectorIndexer

In [ ]:
from pyspark.sql.functions import when
before_final_1 = df_before
before_final_1 = before_final_1.withColumn("score",before_final_1["score"].cast(IntegerType()))
before_final_1 = before_final_1.withColumn("new_score", when(before_final_1["score"]==-1, 2).otherwise(before_final_1["score"]))
before_final_1.select('new_score').distinct().show()


+---------+
|new_score|
+---------+
|        1|
|        2|
|        0|
+---------+



In [ ]:
training_df, testing_df = before_final_1.randomSplit([0.7, 0.3], seed=0)

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

rf = RandomForestClassifier(labelCol='new_score', featuresCol='tfidf', predictionCol='prediction')
pipe2 = Pipeline(stages=[tokenizer, sw_filter, cv]).fit(training_df)
rf_pipe = Pipeline(stages=[pipe2, idf, rf])
modelEvaluator = RegressionEvaluator(labelCol='new_score', metricName='mse', predictionCol='prediction')

In [ ]:
grid = ParamGridBuilder().\
    addGrid(rf.numTrees, [5,10,15,20]).\
    addGrid(rf.maxDepth, [5,10]).\
    addGrid(rf.impurity, ['gini','entropy']).\
    build()
rf_all_models = []
for j in range(len(grid)):
  print("Fitting model {}".format(j+1))
  model = rf_pipe.fit(training_df, grid[j])
  rf_all_models.append(model)
  # estimate the accuracy of each of them:
rf_MSE = []
for i in rf_all_models:
  rf_MSE.append(modelEvaluator.evaluate(i.transform(testing_df)))

Fitting model 1
Fitting model 2
Fitting model 3
Fitting model 4
Fitting model 5
Fitting model 6
Fitting model 7
Fitting model 8
Fitting model 9
Fitting model 10
Fitting model 11
Fitting model 12


In [ ]:
rf_best_mse = min(rf_MSE)
rf_best_mse_index = rf_MSE.index(rf_best_mse)
print('Best mse:', rf_best_mse)
print('The index:', rf_best_mse_index)

Best mse: 0.5464796877258927
The index: 7


In [ ]:
rf_param_dict = rf_all_models[rf_best_mse_index].stages[-1].extractParamMap()

rf_sane_dict = {}
for k, v in rf_param_dict.items():
  rf_sane_dict[k.name] = v

rf_numTrees_par = rf_sane_dict["numTrees"]
rf_maxDepth_par = rf_sane_dict["maxDepth"]
rf_impurity_par = rf_sane_dict["impurity"]
print('numTrees:',rf_numTrees_par,'\nmaxDepth:',rf_maxDepth_par, '\nimpurity:',rf_impurity_par)

numTrees: 10 
maxDepth: 10 
impurity: entropy


In [ ]:
rf = RandomForestClassifier(labelCol='new_score', featuresCol='tfidf', predictionCol='prediction', 
                            numTrees=rf_numTrees_par, maxDepth=rf_maxDepth_par, impurity=rf_impurity_par)
best_pipe = Pipeline(stages=[pipe2, idf, rf])

In [ ]:
best_model = best_pipe.fit(training_df)
best_pred = best_model.transform(testing_df)

In [ ]:
rf_mse = round(modelEvaluator.evaluate(best_pred),2)
print('The best MSE score is',rf_mse)

The best MSE score is 0.55


In [ ]:
rf_model = best_model.stages[-1]
rf_model.featureImportances

SparseVector(17290, {0: 0.0015, 2: 0.0002, 3: 0.0009, 4: 0.0082, 7: 0.0007, 8: 0.0075, 9: 0.0126, 10: 0.0075, 12: 0.0009, 15: 0.017, 16: 0.0007, 18: 0.0005, 20: 0.0013, 21: 0.0, 22: 0.006, 23: 0.0002, 25: 0.0, 30: 0.0602, 32: 0.0007, 33: 0.0021, 38: 0.0016, 40: 0.0005, 42: 0.0072, 44: 0.0002, 50: 0.0005, 53: 0.0008, 54: 0.0159, 55: 0.0285, 58: 0.0142, 60: 0.0007, 65: 0.0328, 67: 0.0005, 69: 0.0002, 71: 0.0673, 75: 0.0323, 77: 0.0, 79: 0.0472, 80: 0.0629, 83: 0.0061, 88: 0.0013, 93: 0.009, 95: 0.0003, 106: 0.0194, 111: 0.0001, 113: 0.0006, 129: 0.0003, 135: 0.0001, 150: 0.0045, 153: 0.0004, 155: 0.0002, 159: 0.0171, 160: 0.0001, 162: 0.0003, 176: 0.0049, 185: 0.0259, 189: 0.0001, 200: 0.0002, 201: 0.0, 205: 0.0011, 212: 0.0013, 213: 0.0108, 217: 0.0081, 218: 0.0001, 222: 0.0027, 227: 0.0005, 228: 0.0101, 230: 0.0022, 233: 0.0, 238: 0.0102, 243: 0.0008, 248: 0.0, 250: 0.0004, 252: 0.0005, 255: 0.0002, 257: 0.0437, 260: 0.0087, 262: 0.0003, 263: 0.0102, 268: 0.0001, 277: 0.0001, 294: 0.00

In [ ]:
vocabulary = best_model.stages[0].stages[-1].vocabulary
importance = best_model.stages[-1].featureImportances
print("num importance:", len(importance))
 
featureimportance_before_df_pd = pd.DataFrame({'word': vocabulary, 'importance': importance})
featureimportance_before_df_pd.head()

num importance: 17290


,word,importance
0,biden,0.001452
1,trump,0.000000
2,joe,0.000186
3,vote,0.000881
4,amp,0.008184


In [ ]:
featureimportance_before_df_pd = featureimportance_before_df_pd.sort_values('importance')
rf_before_df_neg = featureimportance_before_df_pd.head(10)

In [ ]:
featureimportance_before_df_pd = featureimportance_before_df_pd.sort_values('importance', ascending=False)
rf_before_df_pos = featureimportance_before_df_pd.head(10)

In [ ]:
display(rf_before_df_neg)
display(rf_before_df_pos)

,word,importance
8644,instruction,0.0
11514,kellie,0.0
11515,successor,0.0
11516,ratified,0.0
11517,untraditional,0.0
11518,landlords,0.0
11519,arklamiss,0.0
11520,potheadjailed,0.0
11521,agreeable,0.0
11522,await,0.0


,word,importance
71,black,0.067261
80,live,0.062916
30,new,0.060247
79,love,0.047209
257,stupid,0.043715
65,rudy,0.032765
75,sure,0.032303
55,great,0.028517
185,shit,0.025925
106,important,0.019378


### After

In [ ]:
after_final_1 = tweets_after
after_final_1 = after_final_1.withColumn("score",after_final_1["score"].cast(IntegerType()))
after_final_1 = after_final_1.withColumn("new_score", when(after_final_1["score"]==-1, 2).otherwise(after_final_1["score"]))
after_final_1.select('new_score').distinct().show()

+---------+
|new_score|
+---------+
|        1|
|        2|
|        0|
+---------+



In [ ]:
training_df, testing_df = after_final_1.randomSplit([0.7, 0.3], seed=0)

In [ ]:
rf_after = RandomForestClassifier(labelCol='new_score', featuresCol='tfidf', predictionCol='prediction')
pipe4 = Pipeline(stages=[tokenizer, sw_filter, cv]).fit(training_df)
rf_pipe_after = Pipeline(stages=[pipe4, idf, rf])
modelEvaluator = RegressionEvaluator(labelCol='new_score', metricName='mse', predictionCol='prediction')

In [ ]:
grid = ParamGridBuilder().\
    addGrid(rf.numTrees, [5,10,15,20]).\
    addGrid(rf.maxDepth, [5,10]).\
    addGrid(rf.impurity, ['gini','entropy']).\
    build()
rf_all_models_after = []
for j in range(len(grid)):
  print("Fitting model {}".format(j+1))
  model = rf_pipe_after.fit(training_df, grid[j])
  rf_all_models_after.append(model)
  # estimate the accuracy of each of them:
rf_MSE_after = []
for i in rf_all_models_after:
  rf_MSE_after.append(modelEvaluator.evaluate(i.transform(testing_df)))

Fitting model 1
Fitting model 2
Fitting model 3
Fitting model 4
Fitting model 5
Fitting model 6
Fitting model 7
Fitting model 8
Fitting model 9
Fitting model 10
Fitting model 11
Fitting model 12
Fitting model 13
Fitting model 14
Fitting model 15
Fitting model 16


In [ ]:
rf_best_mse_after = min(rf_MSE_after)
rf_best_mse_index_after = rf_MSE_after.index(rf_best_mse_after)
print('Best mse:', rf_best_mse_after)
print('The index:', rf_best_mse_index_after)

Best mse: 0.5441547518923465
The index: 7


In [ ]:
rf_param_dict_after = rf_all_models_after[rf_best_mse_index_after].stages[-1].extractParamMap()

rf_sane_dict_after = {}
for k, v in rf_param_dict_after.items():
  rf_sane_dict_after[k.name] = v

rf_numTrees_par_after = rf_sane_dict_after["numTrees"]
rf_maxDepth_par_after = rf_sane_dict_after["maxDepth"]
rf_impurity_par_after = rf_sane_dict_after["impurity"]
print('numTrees:',rf_numTrees_par_after,'\nmaxDepth:',rf_maxDepth_par_after, '\nimpurity:',rf_impurity_par_after)

numTrees: 10 
maxDepth: 10 
impurity: entropy


In [ ]:
rf_after = RandomForestClassifier(labelCol='new_score', featuresCol='tfidf', predictionCol='prediction', 
                                  numTrees=rf_numTrees_par_after, maxDepth=rf_maxDepth_par_after, impurity=rf_impurity_par_after)
best_pipe_after = Pipeline(stages=[pipe4, idf, rf])

In [ ]:
best_model_after = best_pipe_after.fit(training_df)
best_pred_after = best_model_after.transform(testing_df)

In [ ]:
rf_mse_after = round(modelEvaluator.evaluate(best_pred_after),2)
print('The best MSE score is',rf_mse_after)

The best MSE score is 0.54


In [ ]:
vocabulary = best_model_after.stages[0].stages[-1].vocabulary
importance = best_model_after.stages[-1].featureImportances
print("num importance:", len(importance))
 
featureimportance_after_df_pd = pd.DataFrame({'word': vocabulary, 'importance': importance})
featureimportance_after_df_pd.head()

num importance: 14603


,word,importance
0,trump,0.001658
1,president,0.000000
2,election,0.003242
3,biden,0.000000
4,donald,0.001281


In [ ]:
featureimportance_after_df_pd = featureimportance_after_df_pd.sort_values('importance')
rf_after_df_neg = featureimportance_after_df_pd.head(10)

In [ ]:
featureimportance_after_df_pd = featureimportance_after_df_pd.sort_values('importance', ascending=False)
rf_after_df_pos = featureimportance_after_df_pd.head(10)

In [ ]:
display(rf_after_df_neg)
display(rf_after_df_pos)

,word,importance
7301,sincerely,0.0
9721,opener,0.0
9722,eviscerated,0.0
9723,lecter,0.0
9724,divine,0.0
9725,612,0.0
9726,1965,0.0
9727,did”,0.0
9728,yro,0.0
9729,ribbon”,0.0


,word,importance
25,new,0.064686
75,sweet,0.053045
147,thanks,0.038509
110,legal,0.037724
171,fake,0.034461
23,win,0.032195
107,best,0.028448
218,hero,0.025640
146,dead,0.021989
252,magnificent,0.021148
